# Add New Tokens 

In [6]:
import os
import re
import json
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import scipy.spatial

import tqdm as notebook_tqdm

In [3]:
sen1 = "클라우드 컴퓨팅은 데이터 저장 및 처리를 위해 원격 서버를 활용하는 현대적인 컴퓨터 기술이다." # G06F
sen2 = "클라우드 기반의 분산 컴퓨팅 시스템은 복잡한 데이터 처리 작업을 효율적으로 수행하기 위해 다수의 컴퓨터 자원을 동시에 사용한다." # G06F, 같은 특허
sen3 = "본 발명은 사용자의 행동을 기반으로 개인화된 광고를 실시간으로 제공하는 인공 지능 기반의 시스템에 관한 것이다." # G06F, 다른 특허
sen4 = "새로운 분광학 기법을 사용하여 화합물의 구조를 식별하고 분석할 수 있다." # G01N
sen5 = "자율 주행 차량은 센서와 인공 지능을 활용하여 도로 상황을 인식하고 판단한다." # B60W
sen6 = "LED 램프는 삽입용 러그 및 바닥의 중앙 절연 콘택트를 갖는 관형 금속 본체를 구비한다." # F21V
sen7 = "이 합성 펩타이드는 특정 단백질과 결합하여 암 세포의 성장을 억제한다."  # C07K
sentences = [sen1, sen2, sen3, sen4, sen5, sen6, sen7]

In [10]:
# Load model from HuggingFace Hub
base_tokenizer = AutoTokenizer.from_pretrained('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
base_model = AutoModel.from_pretrained('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

In [11]:
# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [36]:
for sen in sentences:
    token = base_tokenizer.tokenize(sen)
    print(token)

['클라우드', '컴', '##퓨', '##팅', '##은', '데이터', '저장', '및', '처리를', '위해', '원격', '서버', '##를', '활용하는', '현대', '##적인', '컴퓨터', '기술이', '##다', '.']
['클라우드', '기반의', '분산', '컴', '##퓨', '##팅', '시스템은', '복잡한', '데이터', '처리', '작업을', '효율적으로', '수행', '##하기', '위해', '다수의', '컴퓨터', '자원을', '동시에', '사용한다', '.']
['본', '발명', '##은', '사용자의', '행동을', '기반으로', '개인', '##화된', '광고를', '실시간으로', '제공하는', '인공', '지능', '기반의', '시스템에', '관한', '것이다', '.']
['새로운', '분', '##광', '##학', '기법을', '사용하여', '화합', '##물의', '구조를', '식별', '##하고', '분석', '##할', '수', '있다', '.']
['자율', '주행', '차량은', '센', '##서와', '인공', '지능', '##을', '활용', '##하여', '도로', '상황을', '인식하고', '판단한다', '.']
['LED', '램', '##프는', '삽입', '##용', '러', '##그', '및', '바닥', '##의', '중앙', '절', '##연', '콘', '##택', '##트를', '갖는', '관', '##형', '금속', '본', '##체를', '구비', '##한다', '.']
['이', '합성', '펩', '##타이', '##드는', '특정', '단백질', '##과', '결합', '##하여', '암', '세포', '##의', '성장을', '억제', '##한다', '.']


In [37]:
# Tokenize sentences
encoded_input = base_tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = base_model(**encoded_input)

# Perform pooling. In this case, mean pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

print("5 Sentence embeddings:")
print(sentence_embeddings)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


5 Sentence embeddings:
tensor([[-0.2795, -0.9693,  0.4312,  ...,  1.3973,  0.5144,  0.8678],
        [-0.1564, -0.8299,  0.1879,  ...,  1.2240,  0.7727,  0.2770],
        [ 0.1303, -0.4059,  0.3236,  ...,  0.0760,  0.1354,  0.7479],
        ...,
        [ 0.3881, -1.1212,  0.1156,  ...,  0.2851, -0.0500,  0.5711],
        [-0.0854, -0.4651, -0.2107,  ...,  0.7511, -0.2592, -1.1496],
        [ 0.1757, -0.1680,  0.3617,  ...,  0.6420, -0.8075, -0.3247]])


In [38]:
cosine_sim = torch.nn.functional.cosine_similarity(sentence_embeddings[0].unsqueeze(0), sentence_embeddings[1:])

print(cosine_sim)

tensor([0.8690, 0.4795, 0.3099, 0.2552, 0.2694, 0.1289])


## Vocab

### Make Vocab List

In [39]:
import json
import glob
from collections import Counter

folder_path = 'E:/Documents/2024-1 YBIGTA/KPMG/model/특허_말뭉치/*.json'

# 엔티티 빈도수를 저장할 카운터
entity_counts = Counter()

# 지정된 폴더 내의 모든 JSON 파일을 순회
for file_path in glob.glob(folder_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        # JSON 파일 파싱
        data = json.load(file)
        
        # 각 엔티티의 빈도수 계산
        for patent in data["data"]:
            for row in patent["rows"]:
                for entity in row.get("NE", []):
                    entity_counts[entity["entity"]] += 1
                    
# 결과 출력 (예시: 가장 빈번한 10개 엔티티)
for entity, count in entity_counts.most_common(10):
    print(f"{entity}: {count}")

실시예: 300554
도 1: 146543
제 1: 130623
데이터: 127148
사용자: 125600
제 2: 107115
100: 95263
시스템: 93253
도 2: 88037
도 3: 77858


In [41]:
# 결과 출력 (예시: 가장 빈번한 10개 엔티티)
for entity, count in entity_counts.most_common(10):
    print(f"{entity}: {count}")

# 결과를 딕셔너리 형태로 저장
entity_freq_dict = dict(entity_counts)

실시예: 300554
도 1: 146543
제 1: 130623
데이터: 127148
사용자: 125600
제 2: 107115
100: 95263
시스템: 93253
도 2: 88037
도 3: 77858


In [12]:
# 필요하다면 딕셔너리를 파일로 저장
with open('entity_frequency.json', 'w', encoding='utf-8') as f:
    json.dump(entity_freq_dict, f, ensure_ascii=False, indent=4)

### Add words

In [7]:
# JSON 파일 로드
file = 'E:/Documents/2024-1 YBIGTA/KPMG/model/entity_frequency.json'
with open(file, 'r', encoding='utf-8') as f:
    entity_freq_dict = json.load(f)

# 빈도순으로 딕셔너리 정렬
sorted_entity_freq = sorted(entity_freq_dict.items(), key=lambda item: item[1], reverse=True)

# 특정 빈도 이상의 항목만 선택 (예: 2000번 이상 등장하는 엔티티)
threshold = 2000
filtered_entity_freq = {entity: freq for entity, freq in sorted_entity_freq if freq >= threshold}

# 필터링된 결과를 새로운 단어 목록으로 추가
new_words = list(filtered_entity_freq.keys())

# 숫자를 포함하지 않는 단어만 선택
new_words = [word for word in new_words if not re.search(r'\d', word)]

# 결과 출력
print(len(new_words), new_words[-5:])
print(new_words)

1435 ['카테고리', '손잡이부', '코팅액', '물탱크', '유효한']
['실시예', '데이터', '사용자', '시스템', '일 실시예', '프레임', '형성되어', '제어부', '바람직한', '하우징', '플레이트', '화합물', '문제점', '구체적인', '단말기', '조성물', '이미지', '적어도 하나', '메모리', '케이스', '가능한', '실시 예', '서비스', '반도체', '상태에서', '설명하기', '하나 이상', '가이드', '실시하기', '스위치', '일실시예', '메시지', '네트워크', '웨이퍼', '게이트', '제조방법', '어느 하나', '상부에', '실리콘', '제조된', '에너지', '와이어', '기술적', '베이스', '안테나', 'LED', '회전축', '주파수', '선택된', '트랜지스터', '한 쌍', '프로그램', '달성하기', '스프링', '구성요소', '적어도', '실린더', '디스플레이', '배터리', '컴퓨터', '카메라', '디바이스', '이루어질', '레이저', '파이프', '위치에', '복수개', '케이블', '명세서', '실시형태', '우수한', '테이블', '동작을', '초음파', '추출물', '제조 방법', '외주면', '구조물', '플라즈마', '마스크', '프로세서', '구동부', '혼합물', '샤프트', '하부에', '유전자', '일반적', '단백질', '열교환기', '알루미늄', '서로 다른', '기지국', '두 개', '개구부', '범위 내', '지지부', '예시적인', '지지대', '연결부', '문제점을 해결', '커넥터', '인터페이스', '돌출부', '피스톤', '복수 개', '발명의 실시', '캐리어', '냉각수', '절연층', '테스트', '저장부', '컨텐츠', '기술적 사상', '반도체층', '디스크', '콘크리트', '고분자', '특허청구범위', '조립체', '상이한', '접착제', '프로브', '기술분야', '태양전지', '절연막', '화학식', '프로세스', '여러 가지

In [54]:
# # 서브워드 토크나이징
# new_tokens = [base_tokenizer.tokenize(word) for word in new_words]
# flattened_tokens = [token for sublist in new_tokens for token in sublist]
# print(flattened_tokens)
# print(len(flattened_tokens))

['실시', '##예', '데이터', '사용자', '시스템', '일', '실시', '##예', '프레임', '형성', '##되어', '제어', '##부', '바람직한', '하우', '##징', '플레이', '##트', '화합', '##물', '문제점', '구체적인', '단말기', '조성', '##물', '이미지', '적어도', '하나', '메모리', '케이스', '가능한', '실시', '예', '서비스', '반도체', '상태에서', '설명', '##하기', '하나', '이상', '가이드', '실시', '##하기', '스위', '##치', '일', '##실', '##시', '##예', '메시지', '네트워크', '웨이', '##퍼', '게이트', '제조', '##방법', '어느', '하나', '상', '##부에', '실리콘', '제조', '##된', '에너지', '와이', '##어', '기술적', '베이스', '안', '##테', '##나', 'LED', '회전', '##축', '주파수', '선택', '##된', '트랜', '##지', '##스터', '한', '쌍', '프로그램', '달성하기', '스프링', '구성', '##요소', '적어도', '실린', '##더', '디스플레이', '배터리', '컴퓨터', '카메라', '디', '##바이', '##스', '이루어질', '레이저', '파이', '##프', '위치에', '복수', '##개', '케이블', '명', '##세서', '실시', '##형태', '우수한', '테이블', '동작을', '초음', '##파', '추출', '##물', '제조', '방법', '외주', '##면', '구조', '##물', '플라', '##즈', '##마', '마스크', '프로세', '##서', '구', '##동부', '혼합', '##물', '샤', '##프트', '하', '##부에', '유전자', '일반적', '단백질', '열', '##교환', '##기', '알루미늄', '서로', '다른', '기지', '##국', '두', '개', '

In [12]:
# 기존 어휘집에 없는 워드만 필터링하여 추가
existing_vocab = base_tokenizer.get_vocab()
print(len(existing_vocab))

filtered_tokens = [token for token in new_words if token not in existing_vocab]
num_added_toks = base_tokenizer.add_tokens(filtered_tokens)
print(f'Added {num_added_toks} new tokens')

40000
Added 1212 new tokens


In [13]:
# 모델의 토큰 임베딩 크기 조정
base_model.resize_token_embeddings(len(base_tokenizer))

Embedding(41212, 768)

## Check New Tokenizer

In [14]:
# 갱신
model = base_model
tokenizer = base_tokenizer

In [15]:
for sen in sentences:
    token = tokenizer.tokenize(sen)
    print(token)

['클라우드', '컴퓨팅', '은', '데이터 ', '저장', '및', '처리를', '위해', '원격', '서버', '##를', '활용하는', '현대', '##적인', '컴퓨터', '기술이', '##다', '.']
['클라우드', '기반의', '분산', '컴퓨팅', '시스템은', '복잡한', '데이터 ', '처리', '작업을', '효율적', '으로', '수행하기', '위해', '다수의', '컴퓨터', '자원을', '동시에', '사용한다', '.']
['본', '발명', '##은', '사용자의', '행동을', '기반으로', '개인', '##화된', '광고를', '실시간으로', '제공하는', '인공', '지능', '기반의', '시스템에', '관한', '것이다', '.']
['새로운', '분광학', '기법을', '사용하여', '화합물', '의', '구조를', '식별', '##하고', '분석', '##할', '수', '있다', '.']
['자율', '주행', '차량은', '센서', '와', '인공', '지능', '##을', '활용', '##하여', '도로', '상황을', '인식하고', '판단한다', '.']
['LED', '램프', '는', '삽입', '##용', '러', '##그', '및', '바닥', '##의', '중앙', '절', '##연', '콘택트', '를', '갖는', '관', '##형', '금속', '본', '##체를', '구비', '##한다', '.']
['이', '합성', '펩타이드', '는', '특정', '단백질', '##과', '결합', '##하여', '암', '세포', '##의', '성장을', '억제', '##한다', '.']


In [16]:
# Tokenize sentences
encoded_input = base_tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = base_model(**encoded_input)

# Perform pooling. In this case, mean pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

print("5 Sentence embeddings:")
print(sentence_embeddings)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


5 Sentence embeddings:
tensor([[-0.5209, -0.6556,  0.3016,  ...,  1.9143,  0.6019,  0.8433],
        [-0.4207, -0.7607,  0.3010,  ...,  2.1678,  0.6537,  0.6580],
        [ 0.1303, -0.4059,  0.3236,  ...,  0.0760,  0.1354,  0.7479],
        ...,
        [ 0.1473, -1.3644,  0.5968,  ...,  0.9513, -0.1660,  0.8177],
        [-0.1324, -0.3718, -0.1414,  ...,  1.3349,  0.0157, -0.6849],
        [-0.1522, -0.1365,  0.0687,  ...,  1.2886, -0.6347, -0.5048]])


In [17]:
cosine_sim = torch.nn.functional.cosine_similarity(sentence_embeddings[0].unsqueeze(0), sentence_embeddings[1:])

print(cosine_sim)

tensor([0.8476, 0.4470, 0.2163, 0.2414, 0.2944, 0.1859])


In [68]:
# 저장할 경로 지정
save_directory = "./SBERT-added-patent-vocab"

# 모델과 토크나이저 저장
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('./SBERT-added-patent-vocab\\tokenizer_config.json',
 './SBERT-added-patent-vocab\\special_tokens_map.json',
 './SBERT-added-patent-vocab\\vocab.txt',
 './SBERT-added-patent-vocab\\added_tokens.json',
 './SBERT-added-patent-vocab\\tokenizer.json')

In [ ]:
new_model = AutoModel.from_pretrained(save_directory)
new_tokenizer = AutoTokenizer.from_pretrained(save_directory)

In [91]:
for sen in sentences:
    token = new_tokenizer.tokenize(sen)
    print(token)

['클라우드', '컴', '##퓨', '##팅', '##은', '데이터 ', '저장', '및', '처리를', '위해', '원격', '서버', '##를', '활용하는', '현대', '##적인', '컴퓨터', '기술이', '##다', '.']
['클라우드', '기반의', '분산', '컴', '##퓨', '##팅', '시스템은', '복잡한', '데이터 ', '처리', '작업을', '효율적', '으로', '수행하기', '위해', '다수의', '컴퓨터', '자원을', '동시에', '사용한다', '.']
['본', '발명', '##은', '사용자의', '행동을', '기반으로', '개인', '##화된', '광고를', '실시간으로', '제공하는', '인공', '지능', '기반의', '시스템에', '관한', '것이다', '.']
['새로운', '분', '##광', '##학', '기법을', '사용하여', '화합물', '의', '구조를', '식별', '##하고', '분석', '##할', '수', '있다', '.']
['자율', '주행', '차량은', '센', '##서와', '인공', '지능', '##을', '활용', '##하여', '도로', '상황을', '인식하고', '판단한다', '.']
['LED', '램', '##프는', '삽입', '##용', '러', '##그', '및', '바닥', '##의', '중앙', '절', '##연', '콘택트', '를', '갖는', '관', '##형', '금속', '본', '##체를', '구비', '##한다', '.']
['이', '합성', '펩타이드', '는', '특정', '단백질', '##과', '결합', '##하여', '암', '세포', '##의', '성장을', '억제', '##한다', '.']


In [ ]:
# Repository 생성 & model upload
REPO_NAME = 'SBERT-added-patent-vocab' 
AUTH_TOKEN = 'my_token' 

## Upload to Huggingface Hub
model.push_to_hub(
    REPO_NAME, 
    use_temp_dir=True, 
    token=AUTH_TOKEN
)
tokenizer.push_to_hub(
    REPO_NAME, 
    use_temp_dir=True, 
    token=AUTH_TOKEN
)

### 말뭉치

In [ ]:
raw = pd.read_csv("./KPMG/특허청_KIPRISPlus_특허 영한 코퍼스(말뭉치).csv", encoding='cp949')

def get_training_corpus():
    dataset = raw
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        yield samples["번역문"]

training_corpus = get_training_corpus()

In [101]:
new_tokenizer = new_tokenizer.train_new_from_iterator(training_corpus, 40000)

In [102]:
for sen in sentences:
    token = new_tokenizer.tokenize(sen)
    print(token)

['클라우드', '컴퓨팅', '##은', '데이터', '저장', '및', '처리를', '위해', '원격', '서버를', '활용하는', '현대', '##적인', '컴퓨터', '기술이다', '.']
['클라우드', '기반의', '분산', '컴퓨팅', '시스템은', '복잡한', '데이터', '처리', '작업을', '효율적으로', '수행하기', '위해', '다수의', '컴퓨터', '자원', '##을', '동시에', '사용한다', '.']
['본', '발명은', '사용자의', '행동을', '기반으로', '개인화된', '광고', '##를', '실시간으로', '제공하는', '인공', '지능', '기반의', '시스템에', '관한', '것이다', '.']
['새로운', '분광', '##학', '기법을', '사용하여', '화합물의', '구조를', '식별하고', '분석할', '수', '있다', '.']
['자율', '주행', '차량은', '센서와', '인공', '지능', '##을', '활용하여', '도로', '상황을', '인식하고', '판단한다', '.']
['LED', '램프는', '삽입', '##용', '러그', '및', '바닥의', '중앙', '절연', '콘', '##택', '##트를', '갖는', '관형', '금속', '본체를', '구비한다', '.']
['이', '합성', '펩타이드는', '특정', '단백질과', '결합하여', '암', '세포의', '성장을', '억제한다', '.']


In [103]:
# 모델의 토큰 임베딩 크기 조정
new_model.resize_token_embeddings(len(new_tokenizer))

Embedding(40000, 768)

In [105]:
# Tokenize sentences
encoded_input = new_tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = new_model(**encoded_input)

# Perform pooling. In this case, mean pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

print("5 Sentence embeddings:")
print(sentence_embeddings)

cosine_sim = torch.nn.functional.cosine_similarity(sentence_embeddings[0].unsqueeze(0), sentence_embeddings[1:])

print(cosine_sim)

5 Sentence embeddings:
tensor([[-1.1502, -0.0716, -0.4792,  ...,  0.6476,  0.1023, -1.5410],
        [-0.7441, -0.1808, -0.5832,  ...,  1.1759,  0.9235, -0.4281],
        [-1.0382, -0.2870, -0.4285,  ...,  0.8220, -1.0310, -0.4700],
        ...,
        [-0.1718,  0.6589, -0.7152,  ...,  0.8889, -0.5142,  0.2931],
        [-1.5655, -0.6120, -0.1586,  ...,  0.8594, -0.2568, -1.0684],
        [-1.5413, -0.7801,  0.3405,  ...,  0.5996, -1.0910, -0.1559]])
tensor([0.7909, 0.5402, 0.4848, 0.4376, 0.6392, 0.4886])


In [ ]:
# Repository 생성 & model upload
REPO_NAME = 'SBERT-added-korpat-vocab' 
AUTH_TOKEN = 'my_token'

## Upload to Huggingface Hub
model.push_to_hub(
    REPO_NAME, 
    use_temp_dir=True, 
    token=AUTH_TOKEN
)
tokenizer.push_to_hub(
    REPO_NAME, 
    use_temp_dir=True, 
    token=AUTH_TOKEN
)